#### 多分类问题经常会出现数据集分类分布不均匀的情况，通过设置不同分类的权重是一种解决办法，torch中的损失函数可以设置权重，这篇文章主要说明torch中的损失函数的权重是如何实现的。

    以下为torch官网对torch.nn.CrossEntropyLoss类的一些说明，其中，对于权重只是粗略的描述了一下维度，没有说明具体的计算方式，导致光看这个无法准确的设置权重参数

In [ ]:
import torch
loss_model = torch.nn.CrossEntropyLoss(weight=None, size_average=None, ignore_index=- 100, reduce=None, reduction='mean', label_smoothing=0.0)
loss = loss_model(input_data,target)
'''
输入数据需要是原始的、没有经过标准化的各子类数据，数据的形状需要是如下几个之一：（C）或（N，C）或（N，C，d1,d2,...,dk）,其中，C表示待分类的类别数量之和，当需要对像是图片的像素点进行分类的时候，可以用最后一种数据形状,但是需要满足k≥1
The input is expected to contain raw, unnormalized scores for each class. input has to be a Tensor of size (C)(C) for unbatched input, (minibatch, C)(minibatch,C) or (minibatch, C, d_1, d_2, ..., d_K)(minibatch,C,d 
_1,d_2,..,d_K) with K≥1 for the K-dimensional case. The last being useful for higher dimension inputs, such as computing cross entropy loss per-pixel for 2D images.
'''

'''
目标的形状可以是两种，一种是与input_data的形状保持一致，这个时候需要用one-hot表示，另外一种是用列表的索引号来表示一个label，索引号需要大于等于0，小于最大类别数
The target that this criterion expects should contain either:
Class indices in the range [0, C)[0,C) where CC is the number of classes; if ignore_index is specified, this loss also accepts this class index (this index may not necessarily be in the class range).
'''

'''
权重是一个可选的值，如果要使用，权重参数的形状应该等于类别的数目，也就是说是一个1D的数据
weight (Tensor, optional) – a manual rescaling weight given to each class. If given, has to be a Tensor of size C
'''

        通过查阅其他资料和试验，关于权重的设置，先说结论：当有C个类别需要区分，分别为A1,A2……Ac，其在训练集中对应的数据个数分别为N1,N2,……,Nc，如果想设置权重，使得各个类的分布比例一致，需要将各分类对应的权重设置为除了其本身类别之外的其他所有类别的数量之积，例如，对于A1，需要将其权重分布数值设置为N2*N3*N4*...*Nc，对于A2，需要将其权重设置为N1*N3*N4x…*Nc，其他依次类推。如果让Nt=N1*N2*N3*…*Nc，那所有的分类要达到均匀分布需要使用的权重tensor，应该表达为torch.Tensor([Nt/N1,Nt/N2,Nt/N3,……,Nt/Nc])，实例如下：

In [61]:
import torch 

input_data = torch.Tensor([1,0,0,1,0,0,1,0,1,0,0,0])
label = torch.LongTensor([0,1,2,2])

input_data = input_data.view([1,3,4])
label = label.view([1,4])

print('input_data:\n',input_data,'\n')
print('label:\n',label)

input_data:
 tensor([[[1., 0., 0., 1.],
         [0., 0., 1., 0.],
         [1., 0., 0., 0.]]]) 

label:
 tensor([[0, 1, 2, 2]])


        这里的input_data的形状（1,3,4）用的是（N,C,d1,d2,...,dk）这种模式，其中1表示batch_size是1，3表示结果有3个类别，4表示1个数据由4个东西组成，例如可以看作一张由4个像素点组成的图片，input_data的4个column对应4个像素点，每个像素点有3个类别的计算值，column0对应的label值是0，也就是说其对应的正确分类索引是0，同理column1对应的label值是1，其对应的正确分类索引是1，对column2、column3也是一样。下面实现损失权重参数：

In [62]:
#跟据label来生成权重分布tensor
def get_weight(label):
    '''适用于以索引形式表示的label'''
    label = label.view(label.shape[-1])
    label_list = list(label.data.numpy())
    weight_dict = {}
    for i in label_list:
        num=0
        if i not in weight_dict:
            for j in label_list:
                if j==i:
                    num+=1
            weight_dict[i] = num
    tmp = 1
    for value in weight_dict.values():
        tmp = tmp*value
    weight = torch.Tensor([tmp/value for value in weight_dict.values()])
    
    print('生成的权重分布为：',weight)
    print('各个分类对应的数据数量：',weight_dict)
    
    return weight,weight_dict

In [63]:
weight,_ = get_weight(label)
print('\n')

生成的权重分布为： tensor([2., 2., 1.])
各个分类对应的数据数量： {0: 1, 1: 1, 2: 2}




In [64]:
#根据权重分布生成损失函数实例
criterion = torch.nn.CrossEntropyLoss(weight)
loss = criterion(input_data,label)
print(loss)

tensor(1.1707)


#### 接下来详细分析计算过程
    input_data中的column1代表第1个数据的损失，对应的正确分类索引为第0类：
    loss1 = $-log_e^{e^1/(e^1+e^0+e^1)}$ =0.861994   
    input_data中的column2代表第2个数据的损失，对应的正确分类索引为第1类：
    loss2 = $-log_e^{e^0/(e^0+e^0+e^0)}$ =1.098612  
    input_data中的column3代表第3个数据的损失，对应的正确分类索引为第2类：
    loss3 = $-log_e^{e^0/(e^0+e^1+e^0)}$ =1.551444   
    input_data中的column4代表第4个数据的损失，对应的正确分类索引为第2类：
    loss4 = $-log_e^{e^0/(e^1+e^0+e^0)}$ =1.551444        
     
        torch.nn.CrossEntropyLoss会默认对多个损失值取平均值做为最终的输出结果，当没有设置权重分布时：   
        final_loss = (loss1*1+loss2*1+loss3*1+loss4*1)/(1+1+1+1)
    
        可以看到，因为input_data的column3和4的正确类别索引都是2，而column1和2的正确索引是0和1，所以在最后得到final_loss中，分类2的占比是比分类1和0的占比要高，数据的分布这个时候是不均匀的，如果把这种情况极端化，分类0和1仍然各自只有1个数据，但是分类2变成10000个数据，这个时候final_loss就变成基本只由分类2的loss组成的了，下一次更新的时候，模型肯定会尽量往最有利于分类2正确区分的方向更新参数，以保证final_loss能最大程度降低，从而导致忽视分类0和1，使得这两个类别的识别率一直很低。   
    
        为了解决因为数据集中样本类别不均匀，影响模型参数更新方向的问题，我们设置权重分布，设置的方法按照上面说的形式，设置完权重之后的最终损失计算方式为：
        final_loss = (loss1*2+loss2*2+loss3*1+loss4*1)/(2+2+1+1)=1.1707
    
        按照上面所说的原则，这个例子中的label有三类，分别是0、1、2，通过统计，我们知道第0类有1个数据，第1类有1个数据，第2类有2个数据，所以所有分类的数据的乘积为1*1*2=2，因此对应到各个分类应该给与的权重值应该为：第0类的权重为2(各类数据数量的乘积)/1(该类别数据的数量)=2，第1类的权重为2/1=2，第2类的权重为2/2=1。所以传给torch.nn.CrossEntropyLoss的权重tensor应该为tensor([2,2,1]),具体计算时，因为input_data中的column1的正确分类是0，所以loss1应该乘以第0类的权重值2，column2的正确分类是1，所以loss2应该乘以第1类的权重值2，column3的正确分类是2，所以loss3应该乘以第2类的权重值1，column4的正确分类是2，所以loss4应该乘以第2类的权重值1，最后需要将4个loss和权重值相乘的和除以4个对应的权重值之和，得到最终的损失值。
    
        最后，当数据涉及的分类比实际的分类少的时候，没有涉及到的分类的权重值不参与计算，比如其他batch_size所包含的数据里有分类2，但是在我们上面举例的这一个batch_size里的数据column1、2、3、4的正确分类只有0和1，例如4个column的分类分别是0、1、0、1，此时我们针对全局设置的权重分布还是包含3个类别的（2，2，1），此时，我们举例的这一个batch_size的最终损失应该等于：
        final_loss = (loss1*2+loss2*2+loss3*2+loss4*2)/(2+2+2+2)

#### 总的来说，torch.nn.CrossEntropyLoss对于权重平衡是思路是，保证每一个batch_size内涉及到的分类的数据能达到均衡的状态，所以对于我们使用的时候来说，在把数据分配到batch_size的时候，需要让每个batch_size内尽量把各类数据都包含进去，而不能是一个batch_size全是某一个类别的数据，而另一个batch_size内没有这个数据，这种情况可能会导致权重设置失效，在创建DataLoader的时候，设置shuffle为True能大概率避免这个问题。